In [ ]:
import pandas as pd

hate = pd.read_csv("all_examples_hate.csv", sep = "\t")
nonhate = pd.read_csv("all_examples_nonhate.csv", sep = "\t")

In [15]:
hate['label'] = 1
nonhate['label'] = 0

combined = pd.concat([hate, nonhate])

combined_file_path = "all_examples.csv"
combined.to_csv(combined_file_path, sep="\t", index=False)

In [18]:
guideline_counts= combined['guideline'].value_counts()
guideline_counts

guideline
hatecrime victim                                       367
violent speech                                         281
spread virus                                           252
curse profane                                          248
deviating norm                                         245
attacking intellectual capability                      238
attacking derogatory term                              236
criminal                                               234
less than adequate                                     233
better worse than                                      230
attack concept associated protected characteristics    229
curse sexual                                           228
explicit exclusion                                     228
disgust repulsion                                      221
insects                                                218
subhumanity                                            217
attacking appearance                          

# add gpt to test

In [20]:
import pandas as pd

test = pd.read_csv("/data/jzheng36/HateModerate/fine_tune/datasets/testing/hatemoderate_test.csv", sep = "\t")
gpt = pd.read_csv("all_examples_hate.csv.csv", sep = "\t")

gpt = gpt[gpt['dataset'] == 'gpt']

columns_to_keep = ['example_id', 'text', 'guideline', 'labels']
gpt = gpt[columns_to_keep]

test_updated = pd.concat([test, gpt], ignore_index=True)

test_updated.to_csv("/data/jzheng36/HateModerate/fine_tune/datasets/testing/hatemoderate_test_updated.csv", sep="\t", index=False)

In [4]:
def process_df(df):
    # Group by 'guideline' and count the number of rows without 'gpt' in 'dataset'
    counts = df[df['dataset'] != 'gpt'].groupby('guideline').size()

    # Filter rows
    for guideline, count in counts.items():
        if count >= 50:
            # Remove rows with 'gpt'
            df = df[~((df['guideline'] == guideline) & (df['dataset'] == 'gpt'))]
        else:
            # Reduce the number of rows including 'gpt' to 50
            total_to_keep = 50
            non_gpt_count = df[(df['guideline'] == guideline) & (df['dataset'] != 'gpt')].shape[0]
            gpt_to_keep = total_to_keep - non_gpt_count
            # If there are already less than 50 rows, keep them as is
            if gpt_to_keep > 0:
                # Randomly select gpt rows to keep
                gpt_rows = df[(df['guideline'] == guideline) & (df['dataset'] == 'gpt')]
                gpt_rows_kept = gpt_rows.sample(min(gpt_to_keep, len(gpt_rows)), random_state=1)
                # Keep only the selected gpt rows and all non-gpt rows
                df = df[(df['guideline'] != guideline) | (df['dataset'] != 'gpt') | (df.index.isin(gpt_rows_kept.index))]
    
    return df

processed_hate = process_df(hate)
processed_hate_file = "all_examples_hate.csv"

processed_hate.to_csv(processed_hate_file, sep="\t", index=False)

In [31]:
import pandas as pd

# Load the test dataset
test = pd.read_csv("/data/jzheng36/HateModerate/fine_tune/datasets/testing/hatemoderate_test.csv", sep="\t")

# Filter for hate (labels = 1) and non-hate (labels = 0)
hate_test = test[test['labels'] == 1]
nonhate_test = test[test['labels'] == 0]

# Save the filtered datasets to new CSV files
hate_test.to_csv("/data/jzheng36/hatemoderate/hatemoderate/step2/hatemoderate_test_hate.csv", sep="\t", index=False)
nonhate_test.to_csv("/data/jzheng36/hatemoderate/hatemoderate/step2/hatemoderate_test_nonhate.csv", sep="\t", index=False)


In [30]:
ids_to_remove.count()

1